In [4]:
# --- Install necessary packages ---
!pip install flask joblib nest_asyncio --quiet

# --- Import libraries ---
from flask import Flask, request, jsonify, render_template_string
import pandas as pd
import joblib
import nest_asyncio
import threading
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# --- Allow Flask to run in Jupyter Notebook ---
nest_asyncio.apply()

# --- Load and prepare the dataset ---
df = pd.read_csv('cleaned_ad.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour
df.drop(['Ad Topic Line', 'City', 'Timestamp'], axis=1, inplace=True)

X = df.drop('Clicked on Ad', axis=1)
y = df['Clicked on Ad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

numerical_features = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Hour']
categorical_features = ['Country', 'Gender']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

pipeline.fit(X_train, y_train)
joblib.dump(pipeline, 'ad_click_model.pkl')

# --- Define the Flask App ---
app = Flask(__name__)
model = joblib.load('ad_click_model.pkl')

# --- HTML Form Template (Improved with Bootstrap & Try Again button) ---
html_form = """
<!DOCTYPE html>
<html lang="en">
<head>
    <title>Ad Click Predictor</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
    <div class="container mt-5">
        <div class="row justify-content-center">
            <div class="col-md-6">
                <div class="card shadow p-4">
                    <h2 class="card-title text-center mb-4">Ad Click Predictor</h2>
                    <form action="/predict" method="POST">
                        <div class="mb-3">
                            <label class="form-label">Age</label>
                            <input type="number" class="form-control" name="Age" required>
                        </div>
                        <div class="mb-3">
                            <label class="form-label">Daily Time Spent on Site</label>
                            <input type="number" class="form-control" name="Daily Time Spent on Site" step="0.1" required>
                        </div>
                        <div class="mb-3">
                            <label class="form-label">Area Income</label>
                            <input type="number" class="form-control" name="Area Income" step="0.1" required>
                        </div>
                        <div class="mb-3">
                            <label class="form-label">Daily Internet Usage</label>
                            <input type="number" class="form-control" name="Daily Internet Usage" step="0.1" required>
                        </div>
                        <div class="mb-3">
                            <label class="form-label">Hour (0-23)</label>
                            <input type="number" class="form-control" name="Hour" required>
                        </div>
                        <div class="mb-3">
                            <label class="form-label">Country</label>
                            <input type="text" class="form-control" name="Country" required>
                        </div>
                        <div class="mb-3">
                            <label class="form-label">Gender</label>
                            <select class="form-select" name="Gender" required>
                                <option value="">Choose...</option>
                                <option value="Male">Male</option>
                                <option value="Female">Female</option>
                            </select>
                        </div>
                        <button type="submit" class="btn btn-primary w-100">Predict</button>
                    </form>
                    {% if prediction_text %}
                    <div class="alert alert-info mt-4 text-center">
                        <strong>{{ prediction_text }}</strong>
                    </div>
                    <div class="text-center mt-3">
                        <a href="/" class="btn btn-outline-secondary">Try Again</a>
                    </div>
                    {% endif %}
                </div>
            </div>
        </div>
    </div>
</body>
</html>
"""

# --- Home route (empty form, no prediction shown) ---
@app.route('/')
def home():
    return render_template_string(html_form)

# --- Prediction route (displays result after form is submitted) ---
@app.route('/predict', methods=['POST'])
def predict():
    try:
        form_data = request.form.to_dict()
        input_df = pd.DataFrame([form_data])

        # Convert data types
        input_df['Age'] = int(form_data['Age'])
        input_df['Daily Time Spent on Site'] = float(form_data['Daily Time Spent on Site'])
        input_df['Area Income'] = float(form_data['Area Income'])
        input_df['Daily Internet Usage'] = float(form_data['Daily Internet Usage'])
        input_df['Hour'] = int(form_data['Hour'])

        prediction = model.predict(input_df)[0]
        result = "Yes" if prediction else "No"
        return render_template_string(html_form, prediction_text=f"Ad Clicked: {result}")
    except Exception as e:
        return render_template_string(html_form, prediction_text=f"Error: {e}")

# --- API route (JSON-based prediction) ---
@app.route('/api/predict', methods=['POST'])
def api_predict():
    try:
        json_data = request.get_json()
        input_df = pd.DataFrame([json_data])
        prediction = model.predict(input_df)[0]
        return jsonify({'clicked': bool(prediction)})
    except Exception as e:
        return jsonify({'error': str(e)}), 400

# --- Start Flask in background thread (for Jupyter) ---
def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()


127.0.0.1 - - [28/Apr/2025 08:24:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 08:24:55] "GET /favicon.ico HTTP/1.1" 404 -


 * Serving Flask app '__main__'
 * Debug mode: off
